<a href="https://colab.research.google.com/github/jayinc/LLMs4Production/blob/main/semantic_search_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download en_core_web_sm
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install openai==0.27.7

2023-11-16 22:47:51.563200: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 22:47:51.563268: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 22:47:51.563309: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-16 22:47:53.266156: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install gradio

In [ ]:
!pip install -U sentence-transformers rank_bm25

In [ ]:
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
# import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

Read the data for Paris Hotels

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/hamzafarooq/maven-mlsystem-design-cohort-1/main/data/paris_02_11_23.csv')

In [ ]:
df.shape

(28694, 18)

In [ ]:
df.name.value_counts()

Hotel d'Angleterre, Saint Germain des Pres              960
InterContinental Paris - Le Grand                       800
Hotel Marceau Champs Elysees                            800
Sonder Atala Champs-Élysées                             800
Hotel Dauphine Saint Germain                            800
                                                       ... 
Hotel B55                                                94
Les Jardins du Faubourg                                  78
Ibis Styles Paris Meteor Avenue de la Porte d'Italie     62
Hotel Maxim Folies                                       54
Mama Shelter Paris West                                  40
Name: name, Length: 156, dtype: int64

In [ ]:
df=df.drop_duplicates()

In [ ]:
df.to_csv('/content/drive/MyDrive/Semantic_Search/paris_02_11_23.csv',index=False)

In [ ]:
df.shape

(11990, 18)

In [ ]:
df.name.value_counts()

InterContinental Paris - Le Grand                       80
Okko Hotels Paris Porte de Versailles                   80
Citadines Montmartre Paris                              80
Hotel Montfleuri                                        80
Hotel Cayre                                             80
                                                        ..
Seven Hotel Paris                                       32
Ibis Styles Paris Meteor Avenue de la Porte d'Italie    31
Hotel Du Sentier                                        28
Hotel Maxim Folies                                      27
Mama Shelter Paris West                                 20
Name: name, Length: 156, dtype: int64

In [ ]:
df.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."


In [ ]:

df["combined"] = (
    "title: " + df.title.str.strip()+"; Content: " + df.text.str.strip()
    # +"; desc: "+ df.text.str.strip()
)

In [ ]:
df.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features,combined
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title: A large impersonal place with an on tim...
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title: Good hotel with rude waiter; Content: W...
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title: Fantastic; Content: Absolutely top-notc...
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...","title: Amidst the chaos of Fashion week, their..."
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title: Not worth the effort or money; Content:...


In [ ]:
import re

df_combined = df.copy()

df_combined['combined'] = df_combined['combined'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',str(x)))

def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_combined['combined']= df_combined['combined'].apply(lambda x: lower_case(x))


In [ ]:
embedder = SentenceTransformer('all-mpnet-base-v2')

(…)851d5dd1af673670cdb299753/.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

(…)1af673670cdb299753/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)6e48e851d5dd1af673670cdb299753/README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

(…)48e851d5dd1af673670cdb299753/config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

(…)299753/config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

(…)851d5dd1af673670cdb299753/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)1af673670cdb299753/tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

(…)51d5dd1af673670cdb299753/train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

(…)6e48e851d5dd1af673670cdb299753/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)8e851d5dd1af673670cdb299753/modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")
else:
  print("GPU Found!")
  embedder =  embedder.to('cuda')

GPU Found!


In [ ]:
df_combined.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features,combined
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title a large impersonal place with an on time...
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title good hotel with rude waiter content we w...
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title fantastic content absolutely topnotch ro...
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title amidst the chaos of fashion week their s...
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not worth the effort or money content th...


In [ ]:
sample = df_combined[:10]

In [ ]:
embedder =  embedder.to('cpu')

startTime = time.time()

sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


Execution time in seconds: 4.553866147994995


<ipython-input-20-e009c77fa025>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))


In [ ]:

embedder =  embedder.to('cuda')
startTime = time.time()

sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


Execution time in seconds: 0.25376367568969727


<ipython-input-21-290f30ba95f0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["embedding"] = sample.combined.apply(lambda x: embedder.encode(x))


In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

In [ ]:
sample.to_pickle('/content/drive/MyDrive/Semantic_Search/df.pkl')    #to save the dataframe, df to 123.pkl


In [ ]:
df_with_embedding = pd.read_pickle('/content/drive/MyDrive/Semantic_Search/df.pkl') #to load 123.pkl back to the dataframe df

In [ ]:
df_with_embedding.embedding[0]

In [ ]:
df_with_embedding.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features,combined,embedding
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title a large impersonal place with an on time...,"[0.043491535, -0.012825324, 0.0029111414, 0.07..."
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title good hotel with rude waiter content we w...,"[0.055250518, 0.019289263, -0.0056739766, 0.05..."
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title fantastic content absolutely topnotch ro...,"[-0.014940904, 0.02296867, -0.003618411, 0.038..."
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title amidst the chaos of fashion week their s...,"[-0.0036499638, 0.042207047, -0.0062340726, 0...."
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not worth the effort or money content th...,"[0.0013606999, -0.0038351996, 0.0018078145, 0...."


In [ ]:
query = 'Not worth the effort or money + This hotel is not worth the effort or the price'

query_embedding = embedder.encode(query,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df_with_embedding.combined[0]

'title a large impersonal place with an on time check in problem and other issues content if you are looking for a huge grand hotel experience this place may be for you but i found it to be impersonal and the staff lacking warmth and sometimes manners they seem to avoid engagement with guests whenever possible add that to the fact that each morning and evening there are 2 or 3 bus loads of tour groups gathering in the lobby and right outside the hotel when i encountered this on my first evening and needed a taxi i asked the doorman whether he could get me a taxi or whether i should order an uber he simply nodded and walked away so i ordered an uber\n\nthe checkin counter must be understaffed as i had to wait in a long line for about 15 minutes only to be told that my room was not ready official checkin is 2pm but my room was not ready until 5pm i should have taken seriously the many previous tripadvisor reviewers who had the same experience they gave me a drink voucher in the bar as co

In [ ]:
# corpus_embeddings = embedder.encode(df_with_embedding.combined,show_progress_bar=True)

In [ ]:
# df_with_embedding['similarity']=cosine_similarity(corpus_embeddings, query_embedding)

In [ ]:
df_with_embedding['similarity_2'] = df_with_embedding.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

In [ ]:
df_with_embedding

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,...,name,id,description,rating,rating_count,features,combined,embedding,similarity,similarity_2
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,...,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title a large impersonal place with an on time...,"[0.043491535, -0.012825324, 0.0029111414, 0.07...",0.180471,[0.5707]
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,...,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title good hotel with rude waiter content we w...,"[0.055250518, 0.019289263, -0.0056739766, 0.05...",0.144596,[0.45725167]
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,...,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title fantastic content absolutely topnotch ro...,"[-0.014940904, 0.02296867, -0.003618411, 0.038...",0.154253,[0.4877919]
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,...,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title amidst the chaos of fashion week their s...,"[-0.0036499638, 0.042207047, -0.0062340726, 0....",0.151105,[0.47783655]
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,...,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not worth the effort or money content th...,"[0.0013606999, -0.0038351996, 0.0018078145, 0....",0.255166,[0.80690473]
5,862073785,2022-09-26,4,Not quite up to Intercontinental standards,We had a one night stay prior to taking the Eu...,0,/ShowUserReviews-g187147-d207742-r862073785-In...,en,MOBILE,BA9F96090F35C7A562E550825BAB4B32,...,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not quite up to intercontinental standar...,"[0.041726083, 0.033815227, 0.001729757, 0.0653...",0.151502,[0.4790928]
6,861021031,2022-09-20,5,Luxury in the heart of the city,We were very impressed with the intercontinent...,0,/ShowUserReviews-g187147-d207742-r861021031-In...,en,OTHER,CB65D8DF04EEB78702BB03F39BAC6717,...,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title luxury in the heart of the city content ...,"[-0.04693219, -0.017643092, 0.024403231, 0.076...",0.105180,[0.33260918]
7,860421664,2022-09-16,5,Beautiful,Visited for my birthday and everything was jus...,0,/ShowUserReviews-g187147-d207742-r860421664-In...,en,OTHER,9D8328D86D62AFA560367B6F81565010,...,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title beautiful content visited for my birthda...,"[0.0077629383, 0

In [ ]:
# cosine_similarity(corpus_embeddings, query_embedding)

#Entire Data

In [ ]:
startTime = time.time()

df_combined["embedding"] = df_combined.combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


Execution time in seconds: 205.48085594177246


In [ ]:
df_combined.to_pickle('/content/drive/MyDrive/Semantic_Search/entire_data.pkl')

##Embedding upload

In [ ]:
import pandas as pd
df = pd.read_pickle('/content/drive/MyDrive/Semantic_Search/entire_data.pkl') #to load 123.pkl back to the dataframe df

In [ ]:
df.shape

(11990, 20)

In [ ]:
df.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features,combined,embedding
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title a large impersonal place with an on time...,"[0.043491535, -0.012825324, 0.0029111414, 0.07..."
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title good hotel with rude waiter content we w...,"[0.055250518, 0.019289263, -0.0056739766, 0.05..."
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title fantastic content absolutely topnotch ro...,"[-0.014940904, 0.02296867, -0.003618411, 0.038..."
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title amidst the chaos of fashion week their s...,"[-0.0036499638, 0.042207047, -0.0062340726, 0...."
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature...",title not worth the effort or money content th...,"[0.0013606999, -0.0038351996, 0.0018078145, 0...."


In [ ]:
# search through the reviews for a specific product
def search_reviews(df, query, n=5, pprint=True):
    query_embedding = embedder.encode(query,show_progress_bar=True)
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

    results = (
        df.sort_values("similarity", ascending=False)
        .head(n))

    return results

In [ ]:
query = 'close to Louvre and great food nearby but not too expensive'

In [ ]:
results = search_reviews(df,query,15)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
results

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,...,author_username,name,id,description,rating,rating_count,features,combined,embedding,similarity
15436,698261802,2019-08-12,5,Recommend!,Just minutes walk from the Louvre Museums and ...,2,/ShowUserReviews-g187147-d197426-r698261802-Ci...,en,MOBILE,AA83C9E78CC6EC43EDC559DC7C0CDF57,...,Loverofgoodfood1969,Citadines Les Halles Paris,197426,Apart'hotel Citadines Les Halles welcomes you ...,4.0,1426.0,"['roomFeatures_blackout curtains', 'roomFeatur...",title recommend content just minutes walk from...,"[-0.021563573, 0.022654202, 0.0032279596, 0.06...",[0.762306]
10033,858737654,2022-09-06,4,Great hotel close to the Louvre,This was a great location with easy access to ...,1,/ShowUserReviews-g187147-d228694-r858737654-Ho...,en,MOBILE,107486993911F044C7A86C88BD8CBE6C,...,kahnfeldt,Hotel Malte - Astotel,228694,Located in the 2nd district next to the Stock ...,5.0,2285.0,"['roomFeatures_allergy-free room', 'roomFeatur...",title great hotel close to the louvre content ...,"[0.005079792, 0.006933587, 0.0052261045, 0.046...",[0.7389945]
14024,497037862,2017-06-28,5,Fantastic Location and Service- Highly Recomme...,"Best Location in Paris, 3 minutes walking dist...",0,/ShowUserReviews-g187147-d197966-r497037862-Ho...,en,OTHER,5271CD2C9F388D5BCE79D0F5598CC4FF,...,Modhura,Hotel Folkestone Opera,197966,"Welcome to the Folkestone Opera Hotel, your Pa...",4.5,870.0,"['roomFeatures_blackout curtains', 'roomFeatur...",title fantastic location and service highly re...,"[0.0051456545, 0.03332867, 0.009565785, 0.0622...",[0.7292296]
28275,801368976,2021-08-02,5,"Best location, elegant, clean and great rooms.",We spent 5 nights at the lovely Hotel Da Vinci...,1,/ShowUserReviews-g187147-d6675948-r801368976-H...,en,MOBILE,1FEA86D03026BB83FEB43E29EA2B2D0D,...,marydresser,Hotel Da Vinci,6675948,NaN,NaN,NaN,NaN,title best location elegant clean and great ro...,"[-0.04500476, 0.030609902, 0.0011573206, 0.069...",[0.7261322]
16159,727252844,2019-11-18,5,Perfect spot to explore Paris,This was our first visit to Paris and stumbled...,1,/ShowUserReviews-g187147-d635192-r727252844-Re...,en,OTHER,EDB7ED82C206CE0E8692D212CB2B4A4A,...,AlexanderMeerkat,Residence & Spa Le Prince Regent,635192,The Residence & Spa le Prince located in Saint...,4.5,471.0,"['roomFeatures_air conditioning', 'roomFeature...",title perfect spot to explore paris content th...,"[-0.039430726, 0.045125075, 0.009108206, 0.047...",[0.72453666]
6866,806790088,2021-08-28,5,Nice small hotel at an excellent location,"Clean, with a nice design and an excellent loc...",0,/ShowUserReviews-g187147-d278169-r806790088-Ho...,en,OTHER,3C6E94BBBCCA399CFE39851FEF397129,...,do8yb,Hôtel Eugène en Ville,278169,NaN,NaN,NaN,NaN,title nice small hotel at an excellent locatio...,"[-0.013086409, -0.022771467, 0.02151106, 0.072...",[0.71964693]
207,834926510,2022-04-17,5,Best Paris Location,You can’t beat the location of this hotel! Rig...,0,/ShowUserReviews-g187147-d228728-r834926510-Re...,en,MOBILE,58EB5EA5E1A2FCD598F7F7F9D988306A,...,Camper46227911888,Renaissance Paris Vendome Hotel,228728,Bask in the lavish lifestyle of our boutique h...,4.5,1618.0,"['roomFeatures_bathrobes', 'roomFeatures_air c...",title best paris location content you cant bea...,"[-0.0070768404, -0.023154903, 0.011793142, 0.0...",[0.7182149]
6233,227310602,2014-09-06,5,"Fantastic place, good value and well connected","A real find; it's comfortable, quiet, and welc...",1,/ShowUserReviews-g187147-d613301-r227310602-A_...,en,MOBILE,466D8F84569A260E4A3F39A32DDB3AC7,...,bertiebr,A Room In Paris,613301,Right in the city centre of Paris 5 beautiful ...,4.5,237.0,"['roomFeatures_seating area', 'roomFeatures_ir...",title fantastic place good value and well conn...,"[-0.024605073, 0.00506499, 0.024327451, 0.0609...",[0.71735716]
4933,815954462,2021-10-25,5,Great location for romantic trip,"Friendly staff. Great location, relaxed atmosp...",0,/ShowUserReviews-g187147-d22873

In [ ]:
# hlist = []
# for r in results.index:
#     if results.name[r] not in hlist:
#         smalldf = results.loc[results.name == results.name[r]]
#         print('Hotel name: {}'.format(results.name[r]))
#         print('Price per night: {}'.format(results.price_per_night.max()))
#         print('Description: {}'.format(results.description[r]))
#         print('Similarity Score: {}'.format(smalldf.similarity.max()))
#         print('Rating Score: {}'.format(smalldf.rating.max()))

#         print('\n')
#         print('Relevant Reviews:')
#         for s in smalldf.index:
#             print(smalldf.text[s])
#             print('\n')
#         print('xxx---xxx')
#         print('\n')
#         hlist.append(results.name[r])


In [ ]:
# df_combined.shape[1]

In [ ]:
# df = df_combined.copy()

Take all the reviews which are closest to the query, and groupby the hotel name

In [ ]:
def search(query):
  n = 15
  query_embedding = embedder.encode(query)
  df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

  results = (
      df.sort_values("similarity", ascending=False)
      .head(n))

  resultlist = []

  hlist = []
  for r in results.index:
      if results.name[r] not in hlist:
          smalldf = results.loc[results.name == results.name[r]]
          if smalldf.shape[1] > 3:
            smalldf = smalldf[:3]

          resultlist.append(
          {
            "name":results.name[r],
            "score": smalldf.similarity[r][0],
            "rating": smalldf.rating.max(),
            "relevant_reviews": [ smalldf.text[s] for s in smalldf.index]
          })
          hlist.append(results.name[r])
  return resultlist




In [ ]:
search('close to Louvre and great food nearby but not too expensive')

[{'name': 'Citadines Les Halles Paris',
  'score': 0.762306,
  'rating': 4.0,
  'relevant_reviews': ['Just minutes walk from the Louvre Museums and steps to all levels of dining establishments. Could not have been happier when we opened the door to our base rate room to find we had French doors opening to a little balcony with views to die for. It’s a busy city so if you want to be central there plenty of action outside but once the door was closed there was very little noise. Great to have a refrigerator/sink/cutlery/crockery/glassware. Really clean and modern place, fabulous find for the price bracket. Hotel has a laundry for guest use which is handy. Rooms are not serviced, which we liked - no need to prepare your personal items before staff arrive. Second visit and not my last. Staff at check in were excellent/English speaking and we purchased hop on hop off bus tickets there at a reduced price. ']},
 {'name': 'Hotel Malte - Astotel',
  'score': 0.7389945,
  'rating': 5.0,
  'relev

#Building the API

In [ ]:

import gradio as gr

def search(query):
  n = 15
  query_embedding = embedder.encode(query)
  df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1)))

  results = (
      df.sort_values("similarity", ascending=False)
      .head(n))

  resultlist = []

  hlist = []
  for r in results.index:
      if results.name[r] not in hlist:
          smalldf = results.loc[results.name == results.name[r]]
          smallarr = smalldf.similarity[r].max()
          sm =smalldf.rating[r].mean()

          if smalldf.shape[1] > 3:
            smalldf = smalldf[:3]

          resultlist.append(
          {
            "name":results.name[r],
            "description":results.description[r],
            "relevance score": smallarr.tolist(),
            "rating": sm.tolist(),
            "relevant_reviews": [ smalldf.text[s] for s in smalldf.index]
          })
          hlist.append(results.name[r])
  return resultlist

def greet(query):
    bm25 = search(query)
    return bm25

demo = gr.Interface(fn=greet, inputs="text", outputs="json")

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://bb0cbeb8a70ee0568a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
#


In [ ]:
from gradio_client import Client

client = Client("https://3fba686d45d13ad493.gradio.live/")
result = client.predict(
				"Hotel near the Eiffel Tower!",	# str in 'query' Textbox component
				api_name="/predict"
)
print(result)

Loaded as API: https://3fba686d45d13ad493.gradio.live/ ✔
[{'name': 'Hotel Duquesne Eiffel', 'description': 'This cosy hotel offers numerous advantages such as its geographic location, appreciated as much for "leisure" vacations as it is for business meetings. Between "les Invalides" and the Eiffel Tower, the Duquesne Eiffel Hotel is located in one of Paris\' most pleasant boroughs on the famous left bank of the Seine. The team welcomes you with a smile 24/7, while taking all the necessary measures to ensure that your stay is a pleasant one.', 'relevance score': 0.8061095476150513, 'rating': 4.5, 'relevant_reviews': ['The photos for this hotel do not do it justice. I have a front facing room with a view of the Eiffel Tower and it’s just spectacular for what it cost. You don’t need the Shangri La with this hotel. Very friendly staff with quick check in. I was supposed to be here with my husband but his passport didn’t have long enough (make sure you have at least 3 months if you’re comin

In [ ]:
result

[{'name': 'Hotel Duquesne Eiffel',
  'description': 'This cosy hotel offers numerous advantages such as its geographic location, appreciated as much for "leisure" vacations as it is for business meetings. Between "les Invalides" and the Eiffel Tower, the Duquesne Eiffel Hotel is located in one of Paris\' most pleasant boroughs on the famous left bank of the Seine. The team welcomes you with a smile 24/7, while taking all the necessary measures to ensure that your stay is a pleasant one.',
  'relevance score': 0.8061095476150513,
  'rating': 4.5,
  'relevant_reviews': ['The photos for this hotel do not do it justice. I have a front facing room with a view of the Eiffel Tower and it’s just spectacular for what it cost. You don’t need the Shangri La with this hotel. Very friendly staff with quick check in. I was supposed to be here with my husband but his passport didn’t have long enough (make sure you have at least 3 months if you’re coming from the UK) and they still sent up the champag

In [ ]:
!cat /tmp/gradio/tmp0pxsbxyh.json

[{"name": "Hotel Duquesne Eiffel", "description": "This cosy hotel offers numerous advantages such as its geographic location, appreciated as much for \"leisure\" vacations as it is for business meetings. Between \"les Invalides\" and the Eiffel Tower, the Duquesne Eiffel Hotel is located in one of Paris' most pleasant boroughs on the famous left bank of the Seine. The team welcomes you with a smile 24/7, while taking all the necessary measures to ensure that your stay is a pleasant one.", "relevance score": 0.8061095476150513, "rating": 4.5, "relevant_reviews": ["The photos for this hotel do not do it justice. I have a front facing room with a view of the Eiffel Tower and it\u2019s just spectacular for what it cost. You don\u2019t need the Shangri La with this hotel. Very friendly staff with quick check in. I was supposed to be here with my husband but his passport didn\u2019t have long enough (make sure you have at least 3 months if you\u2019re coming from the UK) and they still sent